In [2]:
# Python program to read


# Aqui tá como lemos o arquivo data

# json file
import json
toFile = True #Imprime resultados no arquivo ou na tela
 
# # Opening JSON file
f = open('sample3.json',)
 
# # returns JSON object as
# # a dictionary
data = json.load(f)
f.close()

def printToFile(string,file, end='\n'):
    if toFile:
        message = string+end
        file.write(message)
    else:
        print(string, end=end)

def dataAxes(data): #Retorna os eixos
    #print(dataAxes(data))
    axes = list()
    for axe in data:
        axes.append(axe)
    return axes;

def dataEntities(data, axe): #Retorna as entidades do eixo em específico
    #print(dataEntities(data))
    entities = list()
    for entity in data[axe]:
        entities.append(entity)
    return entities

def entityTables(entityData): #Retorna as chaves das tabelas de determinada de entidade de um eixo
    keys = list()
    for table in entityData:
        for key in table:
            keys.append(key)
    return(keys)

def yearsAvailable(table): #Retorna os anos disponíveis de determinada tabela
    years = list()
    for year in table:
        years.append(year)
    return years


def isSubcategory(table): #Retorna se determinada tabela possui ou não subcategorias
    #print(isSubcategory(data['Eixo Infraestrutura']['escolasUrbanasAlunos'][0]['B6'], i))
    return table['categories']['subcategories']

def totalDetermination(tables, indicator, hasSubcategory, file, subindicator=''):
    totals = []
    years = []
    
    spaces = '    '
    if(subindicator != ''):
#         print('   ',subindicator, ':')
        message = '   '+subindicator+':'
        printToFile(message, file)
        spaces *= 2
    for year in tables:
        years.append(year)
        if (indicator == 'TOTAL'):
            info = tables[year]['indicators'][indicator]
        else:
            info = tables[year]['indicators'][indicator][subindicator]
            
        #{'Sim': 285538, 'Não': 3858, 'Não sabe': 0, 'Não respondeu': 0}
        if(hasSubcategory):
            subcategories = info.values()
            subcategoryIterator = iter(subcategories)
            firstSubcategory = next(subcategoryIterator)
            qty = firstSubcategory.values()
        else:
            qty = info.values()
        
        quantities = qty
        total = 0
        for quantity in quantities:
            total += quantity
        totals.append(total)
    message = spaces+'Total:'
    printToFile(message, file, end=" ")
#     print(spaces,'Total:', end =" "),
    
    n = len(totals)
    for i in range(0, n):
        printToFile(years[i], file, end='')
#         print(years[i], end ='')
        message = '('+str(totals[i])+')'
        printToFile(message, file, end=" ")
#         print(' ({}),'.format(totals[i]), end =" ")
    printToFile('', file)
#     print('')
    
    totals_iters = zip(years, totals)


    totals_list = dict(totals_iters)

    return totals_list
    
def mapCategories(tables, indicator, subindicat = ''):
    subindicators = []
    for year in tables:
        infos = tables[year]['indicators'][indicator]
        if(subindicat != ''):
            infos = tables[year]['indicators'][indicator][subindicat]
        for subindicator in infos:
            if subindicator not in subindicators:
                subindicators.append(subindicator)
    return subindicators
    
def calculateTotal(infoValues):
    total = 0;
    for value in infoValues:
        total += infoValues[value]
    return total

def summaryData(dataPercentage, subindicator, file): #Vai calcular o resumo
    spaces = '    '
    
    if (subindicator != ''): 
        spaces *= 2
    '''
        Esse resumo vai ser calculado olhando para o valor no primeiro ano e o valor do último ano
    '''
    variationsOfEachCategory = {}
    firstData = dataPercentage[0]
    firstYear = ''
    lastYear = ''
    for year in firstData:
        firstYear = year;
    lastData = dataPercentage[-1]
    for year in lastData:
        lastYear = year
    noResume = False
    for category in lastData[lastYear]:
        try:
            variation = lastData[lastYear][category] - firstData[firstYear][category]
            variationsOfEachCategory[category] = variation
        except KeyError:
            message = spaces*2+'Não foi possível analisar com anos anteriores'
            printToFile(message, file)
            #print(spaces*2, 'Não foi possível analisar com anos anteriores')
            noResume = True
    if (not noResume):
        biggerVariation = 0
        categorySelected = ''
        signal = True
        for value in variationsOfEachCategory:
            if(abs(variationsOfEachCategory[value]) > biggerVariation):
                biggerVariation = abs(variationsOfEachCategory[value])
                categorySelected = value
                if variationsOfEachCategory[value] >= 0:
                    signal = True
                else:
                    signal = False
        message = spaces*2+'A principal variação foi de '+categorySelected+' que foi de '+str(round(biggerVariation,2))+'%'
        printToFile(message, file)
        #print(spaces*2, 'A principal variação foi de ',categorySelected,' que foi de ',round(biggerVariation, 2),'%')
        
        

def percentageDetermination(tables, indicator, totals, file, subindicator=''):
    spaces = '    '
    if(subindicator != ''):
        spaces *= 2
    categories = mapCategories(tables,indicator, subindicator) 
    for category in categories:
        message = spaces+category
        printToFile(message, file)
        #print(spaces,category)
        dataPercentage = []
        for year in tables:
            message = spaces*2+'->'
            printToFile(message, file, end=' ')
            #print(spaces*2,'->', end=' ')
            printToFile(year, file, end=': ')
            #print(year, end=': ')
            percentageOfThisYear = {}
            try:
                if(subindicator != ''):
                    info = tables[year]['indicators'][indicator][subindicator][category]
                else:
                    info = tables[year]['indicators'][indicator][category]

                infoValues = info
                total = totals[year]
                for value in infoValues:
                    percentage = round(100*infoValues[value]/total, 2)
                    message = value+':'+str(percentage)+'%'
                    printToFile(message, file, end="| ")
                    #print(value, ':', percentage, '%', end='| ')
                    percentageOfThisYear[value] = percentage;
            except KeyError: 
                printToFile('Não possui', file, end=' ')
                #print('Não possui', end=' ')
            printToFile('\n', file, end='')
            #print('\n', end='')
            dataPercentage.append({year: percentageOfThisYear})
        summaryData(dataPercentage, subindicator, file)

def simplePercentageDetermination(tables, indicator, totals, file, subindicator=''):
    spaces = '    '
    if(subindicator != ''):
        spaces *= 2
    
    dataPercentage = []
    for year in tables:
        message = spaces+'->'
        printToFile(message, file, end= ' ')
        #print(spaces, '->', end=' ')
        printToFile(year, file, end=': ')
        #print(year, end=': ')
        percentageOfThisYear = {}
        try:
            if(subindicator != ''):
                info = tables[year]['indicators'][indicator][subindicator]
            else:
                info = tables[year]['indicators'][indicator]
            infoValues = info
            total = totals[year]
            for value in infoValues:
                percentage = round(100*infoValues[value]/total, 2)
                message = value+':'+str(percentage)+'%'
                printToFile(message, file, end='| ')
                #print(value, ':', percentage, '%', end='| ')
                percentageOfThisYear[value] = percentage;
        except KeyError: 
            printToFile('Não pssui', file, end=' ')
            #print('Não possui', end=' ')
        printToFile('\n', file, end='')
        #print('\n', end='')
        dataPercentage.append({year: percentageOfThisYear})
    summaryData(dataPercentage,subindicator, file)

# Analisando dados de uma tabela que possui subcategorias
def subcategoryTableAnalisys(tables, categoriesAnalysed, file):
    years = yearsAvailable(tables)
    
    table = tables[years[-1]]
    
    hasSubcategory = isSubcategory(table)
        
    indicators = 'indicators'
    
    #print(table['title']) #future write
    printToFile(table['title'], file)
    #print(B62017.keys())
    #print(B62017[indicators].keys())
    for indicator in table[indicators]:
        if indicator not in categoriesAnalysed:
            continue
        message = 'Em termos de '+indicator+':'
        printToFile(message, file)
        if(indicator != 'TOTAL'):
            for subindicator in table[indicators][indicator]:
                if(subindicator == 'Total - Públicas'):
                    continue
                totals = totalDetermination(tables,indicator, hasSubcategory, file, subindicator)
                if(hasSubcategory):
                    percentageDetermination(tables, indicator, totals, file, subindicator)
                else:
                    simplePercentageDetermination(tables, indicator, totals, file, subindicator)
#             break;
        else:
            totals = totalDetermination(tables,indicator, hasSubcategory, file)
            if(hasSubcategory):
                percentageDetermination(tables, indicator, totals, file)
            else:
                simplePercentageDetermination(tables, indicator, totals, file)



categoriesAnalysed = ['TOTAL', 'SEXO', 'DEPENDÊNCIA ADMINISTRATIVA', 'REGIÃO']

# Itera por todos dados de determinada entidade de determinado eixo
axes = dataAxes(data)
for axe in axes:
    print(axe)
    for entity in dataEntities(data, axe):
        print('    ',entity)
        tables = data[axe][entity]
        tablesCodes = entityTables(tables)
        size = len(tables)
        for i in range(0, size):
            table = tables[i][tablesCodes[i]]
            print('        ',table['2019']['title'],'\n')
            pathname = 'resultados/'+axe+'/'+entity+'_'+tablesCodes[i]+'.txt'
            file1 = open(pathname, "w")
            subcategoryTableAnalisys(table, categoriesAnalysed, file1)
            file1.close()
            
    
teste = data[axes[0]][entities[1]]
testeKeys = entityTables(teste)


i = 1
tabelaExemplo = teste[i][testeKeys[i]]

# file1 = open(pathname, "w")
# subcategoryTableAnalisys(tabelaExemplo, categoriesAnalysed, file1)
# file1.close()





Eixo Infraestrutura
     escolasUrbanasAlunos
         TOTAL PARA O INDICADOR B6 – ALUNOS DE ESCOLAS URBANAS, POR TIPO DE COMPUTADOR EXISTENTE NO DOMICÍLIO 

         TOTAL PARA O INDICADOR B10 – ALUNOS DE ESCOLAS URBANAS, POR LOCAL DE ACESSO À INTERNET 

         TOTAL PARA O INDICADOR B11 – ALUNOS DE ESCOLAS URBANAS, POR PRINCIPAL LOCAL DE ACESSO À INTERNET 

     escolasUrbanasProfessores
         TOTAL PARA O INDICADOR B3 – PROFESSORES DE ESCOLAS URBANAS, POR TIPO DE COMPUTADOR EXISTENTE NO DOMICÍLIO 

         TOTAL PARA O INDICADOR B6 – PROFESSORES DE ESCOLAS URBANAS COM ACESSO À INTERNET NO DOMICÍLIO 

     escolasUrbanasEscolas
         TOTAL PARA O INDICADOR D21A – ESCOLAS URBANAS, POR PROGRAMA DE IMPLEMENTAÇÃO DE INFRAESTRUTURA TECNOLÓGICA 

         TOTAL PARA O INDICADOR D25A – ESCOLAS URBANAS, POR VELOCIDADE DA PRINCIPAL CONEXÃO À INTERNET 

Eixo Conteúdos e Recursos Digitais
     escolasUrbanasProfessores
         TOTAL PARA O INDICADOR G1 – PROFESSORES DE ESCOLAS URBANAS

NameError: name 'entities' is not defined